<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2023/blob/main/IsopropanolSynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!python -m pip install importnb --quiet
from importnb import Notebook
with Notebook():
    from che4 import Range, Comp, CompArray, RangeArray, DotDict, d2nt, Props

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 KB 2.6 MB/s eta 0:00:00


In [2]:
import numpy as np
from scipy.optimize import root

In [3]:
#rx1 C3H6 + H2O -> C3H7OH
#rx2 2C3H7OH -> (C3H7)O(C3H7) + H2O
#1. C3H6, 2. (C3H7)O(C3H7), 3. C3H7OH, 4. H2O

e=DotDict()
e.stoic_rx1 = np.array([-1., 0., 1., -1.])
e.stoic_rx2 = np.array([0., 1., -2, 1.])

e.alkeneF = 100.
e.alkeney = np.array([1., 0., 0., 0.])
e.waterF = 95.
e.waterx = np.array([0., 0., 0., 1.])

e.etherx = np.array([0.001, 0.9, 0.09, 0.01])
e.alcoholx = np.array([0.001, 0.01, 0.94, 0.049])

e.etherP = Range(5., 0., e.alkeneF/2)
e.alcoholP = Range(50., 0., e.alkeneF)

e.ext_rx1 = Range(e.alkeneF/2, 0., e.alkeneF)
e.ext_rx2 = Range(e.alkeneF/4, 0., e.alkeneF/2)

In [4]:
x0, x2nt, wrap, x2unk, const=d2nt(e)

{'stoic_rx1': Array([-1.,  0.,  1., -1.], dtype=float64),
 'stoic_rx2': Array([ 0.,  1., -2.,  1.], dtype=float64),
 'alkeneF': Array(100., dtype=float64),
 'alkeney': Array([1., 0., 0., 0.], dtype=float64),
 'waterF': Array(95., dtype=float64),
 'waterx': Array([0., 0., 0., 1.], dtype=float64),
 'etherx': Array([0.001, 0.9  , 0.09 , 0.01 ], dtype=float64),
 'alcoholx': Array([0.001, 0.01 , 0.94 , 0.049], dtype=float64),
 'etherP': Array(5., dtype=float64),
 'alcoholP': Array(50., dtype=float64),
 'ext_rx1': Array(50., dtype=float64),
 'ext_rx2': Array(25., dtype=float64)}

In [5]:
def eqs(d):
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx
    rxns = d.ext_rx1*d.stoic_rx1 + d.ext_rx2*d.stoic_rx2
    prod =  d.etherP*d.etherx + d.alcoholP*d.alcoholx
    return feed + rxns - prod

In [12]:
wrapped_eqs=wrap(eqs)

In [15]:
x=root(wrapped_eqs, x0).x 

In [19]:
x2unk(x)._asdict()

{'etherP': Array(9.2715403, dtype=float64),
 'alcoholP': Array(85.81427397, dtype=float64),
 'ext_rx1': Array(99.90491419, dtype=float64),
 'ext_rx2': Array(9.20252901, dtype=float64)}

In [21]:
const._asdict()

{'stoic_rx1': Array([-1.,  0.,  1., -1.], dtype=float64),
 'stoic_rx2': Array([ 0.,  1., -2.,  1.], dtype=float64),
 'alkeneF': Array(100., dtype=float64),
 'alkeney': Array([1., 0., 0., 0.], dtype=float64),
 'waterF': Array(95., dtype=float64),
 'waterx': Array([0., 0., 0., 1.], dtype=float64),
 'etherx': Array([0.001, 0.9  , 0.09 , 0.01 ], dtype=float64),
 'alcoholx': Array([0.001, 0.01 , 0.94 , 0.049], dtype=float64)}